# GetTemplate Experiments (21/09/2016)

In [1]:
import lsst.meas.astrom as measAstrom
import lsst.daf.base as dafBase
import lsst.pipe.base as pipeBase
import lsst.daf.persistence as dafPersist

import lsst.afw.display.ds9 as ds9
import DiaSourceTools as DSTools

import lsst.afw.table as afwTable
import lsst.afw.geom as afwGeom

import numpy as np

/renoir_data_00/fouchez/lsst/DM/stacks/lsstsw/2016-04-04/lsstsw/miniconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [8]:
import lsst.meas.algorithms.detection as sDet

#DATADIR="/renoir_data_02/jpreyes/lsst_data/CFHTLS_v11/"
DATADIR="/renoir_data_02/jpreyes/lsst_data/CFHTLS_master/"
butler = dafPersist.Butler(DATADIR+"/output")
filter = 'i'
dataId = {'visit': 853737, 'filter':filter , 'ccd':14}
diaSrcs = butler.get("deepDiff_diaSrc", dataId)
calexp = butler.get("calexp", dataId)
diffExp = butler.get("deepDiff_differenceExp", dataId)

In [4]:
import lsst.ip.diffim.getTemplate as getTemplate

In [16]:
dr = butler.dataRef("calexp", dataId=dataId)


In [17]:
gT = getTemplate.GetCoaddAsTemplateTask()
results = gT.run(calexp, dr)

In [41]:
exp = results.exposure
exp.setPsf(calexp.getPsf())
sources = results.sources

In [21]:
ds9.mtv(exp, frame=1)
ds9.mtv(calexp, frame=2)

In [25]:
print calexp.getWcs()

<lsst.afw.image.imageLib.Wcs; proxy of <Swig Object of type 'boost::shared_ptr< lsst::afw::image::Wcs > *' at 0x7f6291b2b3f0> >


In [42]:
import lsst.afw.math as afwMath
config = ImagePsfMatchConfig()
kConfig = config.kernel.active
warper = afwMath.Warper.fromConfig(kConfig.warpingConfig)
templateExposure = warper.warpExposure(calexp.getWcs(), exp, destBBox=calexp.getBBox())


In [37]:
ds9.mtv(templateExposure, frame=3)

In [32]:
import lsst.pex.config as pexConfig
from lsst.ip.diffim.psfMatch import PsfMatchTask, PsfMatchConfigDF, PsfMatchConfigAL
from lsst.meas.algorithms import SourceDetectionTask, getBackground
from lsst.meas.base import SingleFrameMeasurementTask


class ImagePsfMatchConfig(pexConfig.Config):
    """!Configuration for image-to-image Psf matching"""
    kernel = pexConfig.ConfigChoiceField(
        doc="kernel type",
        typemap=dict(
            AL=PsfMatchConfigAL,
            DF=PsfMatchConfigDF
        ),
        default="AL",
    )
    selectDetection = pexConfig.ConfigurableField(
        target=SourceDetectionTask,
        doc="Initial detections used to feed stars to kernel fitting",
    )
    selectMeasurement = pexConfig.ConfigurableField(
        target=SingleFrameMeasurementTask,
        doc="Initial measurements used to feed stars to kernel fitting",
    )

    def setDefaults(self):
        # High sigma detections only
        self.selectDetection.reEstimateBackground = False
        self.selectDetection.thresholdValue = 10.0

        # Minimal set of measurments for star selection
        self.selectMeasurement.doApplyApCorr = "no"
        self.selectMeasurement.algorithms.names.clear()
        self.selectMeasurement.algorithms.names = ('base_SdssCentroid', 'base_PsfFlux', 'base_PixelFlags',
                                                   'base_SdssShape', 'base_GaussianFlux', 'base_SkyCoord')
        self.selectMeasurement.slots.modelFlux = None
        self.selectMeasurement.slots.apFlux = None
        self.selectMeasurement.slots.calibFlux = None


In [48]:
print templateExposure.getPsf()
print exp.getPsf()
templateExposure.setPsf(exp.getPsf())

None
<lsst.afw.detection.detectionLib.Psf; proxy of <Swig Object of type 'boost::shared_ptr< lsst::afw::detection::Psf > *' at 0x7f62922e7cf0> >


In [78]:
import lsst.afw.table as afwTable
from lsst.meas.algorithms.detection import SourceDetectionTask



schema = afwTable.SourceTable.makeMinimalSchema()
table = afwTable.SourceTable.make(schema)

config = SourceDetectionTask.ConfigClass()
config.reEstimateBackground = False

detectionTask = SourceDetectionTask(config=config, schema=schema)
table = afwTable.SourceTable.make(schema)
results = detectionTask.makeSourceCatalog(table=table, exposure=exp , doSmooth=False )
sources = detectionTask.run(table, templateExposure, sigma=4).sources

In [80]:
s = sources[-1]

In [81]:
s.getCentroid()

LogicError: 
  File "include/lsst/afw/table/BaseRecord.h", line 86, in const typename lsst::afw::table::Field<T>::Element* lsst::afw::table::BaseRecord::getElement(const lsst::afw::table::Key<U>&) const [with T = double; typename lsst::afw::table::Field<T>::Element = double]
    Key is not valid (if this is a SourceRecord, make sure slot aliases have been setup). {0}
lsst::pex::exceptions::LogicError: 'Key is not valid (if this is a SourceRecord, make sure slot aliases have been setup).'


In [82]:
import SourceExtractor

ImportError: No module named measurement

In [84]:
import lsst.meas.algorithms as meas_alg

In [85]:
meas_alg.utils.

<module 'lsst.meas.algorithms.utils' from '/renoir_data_00/fouchez/lsst/DM/stacks/lsstsw/2016-04-04/lsstsw/stack/Linux64/meas_algorithms/2016_01.0-8-g252b6e6/python/lsst/meas/algorithms/utils.pyc'>